In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV , train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback, EarlyStopping , ModelCheckpoint
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout


In [37]:
keyword = []
text = []
subcat = []

with open('/content/drive/MyDrive/intern/nlp/allresult2.txt', 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split(' ', 2)  # Splitting on the first two spaces
        if len(parts) == 3:  # Ensure there are three parts: keyword, first part of text, second part of text
            keyword.append(parts[0])
            text.append(parts[1])
            subcat.append(parts[2])

data = pd.DataFrame({'keyword': keyword, 'text': text, 'subcat': subcat})


In [38]:
# turn categorical labels into a numeric
le = LabelEncoder()
y = le.fit_transform(data['keyword'])
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3]


In [39]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(data['text'], y, test_size=0.15, random_state=42)
# print(Train_X)

In [40]:
#converts text into numerical values, specifically into a matrix of TF-IDF features.
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
print(Train_X_Tfidf)   #(row,col    TF-IDF score)
# When a document has only one word, the TF-IDF score for that word in that specific document will be 1.0.

# print(Tfidf_vect.vocabulary_)


  (0, 356)	1.0
  (1, 337)	0.3263678581046755
  (1, 328)	0.3263678581046755
  (1, 288)	0.36890434087673707
  (1, 260)	0.36890434087673707
  (1, 228)	0.36890434087673707
  (1, 211)	0.3263678581046755
  (1, 157)	0.36890434087673707
  (1, 118)	0.36890434087673707
  (2, 337)	0.3263678581046755
  (2, 328)	0.3263678581046755
  (2, 288)	0.36890434087673707
  (2, 260)	0.36890434087673707
  (2, 228)	0.36890434087673707
  (2, 211)	0.3263678581046755
  (2, 157)	0.36890434087673707
  (2, 118)	0.36890434087673707
  (3, 369)	1.0
  (4, 344)	0.2316560662621703
  (4, 319)	0.252483301908755
  (4, 305)	0.5236969653251488
  (4, 296)	0.2618484826625744
  (4, 285)	0.2618484826625744
  (4, 274)	0.2618484826625744
  (4, 264)	0.2618484826625744
  :	:
  (245, 98)	1.0
  (246, 373)	1.0
  (247, 186)	0.7400032589036016
  (247, 121)	0.6726032833788793
  (248, 91)	1.0
  (249, 97)	1.0
  (250, 374)	0.7071067811865475
  (250, 198)	0.7071067811865475
  (251, 219)	0.7071067811865475
  (251, 204)	0.7071067811865475
  (252, 

In [41]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)


Naive Bayes Accuracy Score ->  42.22222222222222


In [42]:
# 1. Preprocess the input text
input_text = " ผู้อนุมัติเอกสารสามารถตรวจสอบความถูกต้องของเนื้อหาและมีสิทธิ์ในการแก้ไขเนื้อหาแล้วส่งต่อผู้พิจารณาเอกสารล าดับถัดไป หรือส่งคืนเอกสารให้ผู้ร่างเอกสารด าเนินการแก้ไข"

# Transform the input text using the same Tfidf_vect object that you used for transforming the training data
input_text_tfidf = Tfidf_vect.transform([input_text])

# 2. Predict using the Naive Bayes model
prediction = Naive.predict(input_text_tfidf)

# The prediction will directly be the label index, so no need to use np.argmax
predicted_label = le.inverse_transform([prediction[0]])  # inverse transform to get the original class label

print(f"Predicted class: {predicted_label[0]}")


Predicted class: 1


In [43]:
Train_X_Tfidf.shape

(255, 389)

In [44]:
print(classification_report(Test_Y, predictions_NB))


              precision    recall  f1-score   support

           0       0.41      0.75      0.53        12
           1       0.18      0.20      0.19        10
           2       1.00      0.50      0.67        12
           3       0.33      0.18      0.24        11

    accuracy                           0.42        45
   macro avg       0.48      0.41      0.41        45
weighted avg       0.50      0.42      0.42        45



In [45]:
# SVM with hyperparameter tuning
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(Train_X_Tfidf, Train_Y)

# Predict using the best model
predictions_SVM = clf.best_estimator_.predict(Test_X_Tfidf)

In [46]:
# 1. Preprocess the input text
input_text = " ผู้อนุมัติเอกสารสามารถตรวจสอบความถูกต้องของเนื้อหาและมีสิทธิ์ในการแก้ไขเนื้อหาแล้วส่งต่อผู้พิจารณาเอกสารล าดับถัดไป หรือส่งคืนเอกสารให้ผู้ร่างเอกสารด าเนินการแก้ไข"

# Transform the input text using the same Tfidf_vect object that you used for transforming the training data
input_text_tfidf = Tfidf_vect.transform([input_text])

# 2. Predict using the Naive Bayes model
prediction = clf.predict(input_text_tfidf)

# The prediction will directly be the label index, so no need to use np.argmax
predicted_label = le.inverse_transform([prediction[0]])  # inverse transform to get the original class label

print(f"Predicted class: {predicted_label[0]}")


Predicted class: 1


In [47]:
print(classification_report(Test_Y,predictions_SVM))

              precision    recall  f1-score   support

           0       0.46      1.00      0.63        12
           1       0.30      0.30      0.30        10
           2       1.00      0.42      0.59        12
           3       0.25      0.09      0.13        11

    accuracy                           0.47        45
   macro avg       0.50      0.45      0.41        45
weighted avg       0.52      0.47      0.42        45



In [48]:
# One-hot encoding for the labels
y_train_encoded = tf.keras.utils.to_categorical(Train_Y)
y_test_encoded = tf.keras.utils.to_categorical(Test_Y)


In [49]:
# Tokenize and pad sequences
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(Train_X)
# turn word to number same word same number
train_sequences = tokenizer.texts_to_sequences(Train_X)
print(train_sequences)  #array len(128)
test_sequences = tokenizer.texts_to_sequences(Test_X)
# print(test_sequences)  arrray len(32)


# same length before training the  data
Train_X_seq = pad_sequences(train_sequences, maxlen=max_len)
Test_X_seq = pad_sequences(test_sequences, maxlen=max_len)



[[42], [11], [11], [12], [13], [43], [44], [45], [46], [7], [18], [19], [8], [3], [9], [8], [47], [48], [6], [49], [50], [51], [14], [52], [4], [20], [21], [15], [5], [53], [22], [54], [55], [56], [57], [58], [2], [59], [23], [60], [61], [62], [14], [63], [64], [2], [24], [65], [66], [25], [67], [2], [68], [26], [27], [20], [69], [70], [27], [9], [15], [28], [1], [71], [72], [73], [74], [7], [75], [16], [29], [76], [30], [77], [78], [79], [80], [81], [3], [31], [7], [82], [14], [83], [84], [18], [30], [5], [85], [32], [86], [87], [4], [5], [88], [33], [89], [3], [34], [6], [34], [35], [90], [91], [92], [93], [8], [94], [95], [10], [1], [96], [97], [98], [1], [5], [99], [100], [16], [101], [17], [102], [1], [103], [104], [10], [105], [17], [106], [12], [107], [36], [108], [23], [16], [6], [109], [37], [110], [111], [38], [28], [112], [37], [17], [113], [6], [12], [3], [114], [115], [33], [19], [116], [117], [118], [39], [119], [2], [120], [121], [1], [32], [122], [1], [38], [40], [123],

In [50]:
# Building the CNN model with added complexity
model = tf.keras.models.Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    Dropout(0.5),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train_encoded.shape[1], activation='softmax')
])


# Load the saved model
newModel = tf.keras.models.load_model('/content/drive/MyDrive/code/best_model_nlp_cnn.h5')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up the ModelCheckpoint callback
checkpoint_path = '/content/drive/MyDrive/code/best_model_nlp4_cnn.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True)

# Train the model with ModelCheckpoint and EarlyStopping callbacks
history = model.fit(Train_X_seq, y_train_encoded, validation_data=(Test_X_seq, y_test_encoded), epochs=100, batch_size= 20, callbacks=[checkpoint, early_stopping], shuffle=True)

# Extracting accuracy and printing
train_accuracies = history.history['accuracy']
valid_accuracies = history.history['val_accuracy']
max_train_accuracy = max(train_accuracies)
max_valid_accuracy = max(valid_accuracies)

print(f"Highest Training Accuracy: {max_train_accuracy*100:.2f}%")
print(f"Highest Validation Accuracy: {max_valid_accuracy*100:.2f}%")

Epoch 1/100
13/13 [==============================] - ETA: 0s - loss: 1.3986 - accuracy: 0.2353
Epoch 1: val_accuracy improved from -inf to 0.22222, saving model to /content/drive/MyDrive/code/best_model_nlp4_cnn.h5
13/13 [==============================] - 3s 110ms/step - loss: 1.3986 - accuracy: 0.2353 - val_loss: 1.3880 - val_accuracy: 0.2222
Epoch 2/100
13/13 [==============================] - ETA: 0s - loss: 1.3927 - accuracy: 0.2667
Epoch 2: val_accuracy improved from 0.22222 to 0.35556, saving model to /content/drive/MyDrive/code/best_model_nlp4_cnn.h5
13/13 [==============================] - 1s 89ms/step - loss: 1.3927 - accuracy: 0.2667 - val_loss: 1.3870 - val_accuracy: 0.3556
Epoch 3/100
13/13 [==============================] - ETA: 0s - loss: 1.3910 - accuracy: 0.2275
Epoch 3: val_accuracy did not improve from 0.35556
13/13 [==============================] - 1s 78ms/step - loss: 1.3910 - accuracy: 0.2275 - val_loss: 1.3869 - val_accuracy: 0.2222
Epoch 4/100
13/13 [===========

In [51]:
# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/code/best_model_nlp4_cnn.h5')

# Evaluate the model on the test data
loss, accuracy = loaded_model.evaluate(Test_X_seq, y_test_encoded, batch_size=3)
print(f"Model Accuracy on Test Data: {accuracy*100:.2f}%")


15/15 [==============================] - 0s 5ms/step - loss: 1.1084 - accuracy: 0.5556
Model Accuracy on Test Data: 55.56%


In [52]:
y_pred_probabilities = model.predict(Test_X_seq) #Get the predictions for the test data
y_pred_labels = np.argmax(y_pred_probabilities, axis=1) # Convert probability distributions to class labels


print(classification_report(Test_Y, y_pred_labels))


2/2 [==============================] - 0s 14ms/step
              precision    recall  f1-score   support

           0       0.44      1.00      0.62        12
           1       0.00      0.00      0.00        10
           2       1.00      0.42      0.59        12
           3       0.62      0.73      0.67        11

    accuracy                           0.56        45
   macro avg       0.51      0.54      0.47        45
weighted avg       0.54      0.56      0.48        45



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
# 1. Preprocess the input text
input_text = " ผู้อนุมัติเอกสารสามารถตรวจสอบความถูกต้องของเนื้อหาและมีสิทธิ์ในการแก้ไขเนื้อหาแล้วส่งต่อผู้พิจารณาเอกสารล าดับถัดไป หรือส่งคืนเอกสารให้ผู้ร่างเอกสารด าเนินการแก้ไข"

# Tokenize and pad
sequence = tokenizer.texts_to_sequences([input_text])
padded_sequence = pad_sequences(sequence, maxlen=max_len)
print(sequence)

# 2. Predict using the model
prediction = model.predict(padded_sequence)
# 3. Decode the prediction

predicted_label_index = np.argmax(prediction, axis=1)[0]
predicted_label = le.inverse_transform([predicted_label_index])

print(f"Predicted class: {predicted_label[0]}")


[[]]
1/1 [==============================] - 0s 22ms/step
Predicted class: 1
